In [251]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lag
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Finding User Purchases").getOrCreate()

In [255]:
df_input = spark.read.options(header='true', InferSchema='true', delimiter=',').csv('amazon_transactions.csv')

In [256]:
df_input.createOrReplaceTempView("transaction")
df_max_sal= spark.sql("select user_id,created_at, _date,rn from "+ " (select user_id,created_at,substring(created_at, 1,10) as _date, row_number() OVER (PARTITION BY user_id ORDER BY created_at ASC) as rn " +" FROM transaction) tmp ")
#df_max_sal.show()

In [257]:
df_date_format= df_max_sal.select(col("user_id"),col("rn"),to_date(col("_date"),"dd-MM-yyyy").alias("date_format_updated"))

In [258]:
df_td= df_date_format.withColumn("time_intertweet", datediff(df_date_format.date_format_updated, lag(df_date_format.date_format_updated, 1).over(Window.partitionBy("user_id").orderBy("date_format_updated"))))

In [259]:
df_td.show()

+-------+---+-------------------+---------------+
|user_id| rn|date_format_updated|time_intertweet|
+-------+---+-------------------+---------------+
|    108|  1|         2020-03-18|           null|
|    108|  2|         2020-03-28|             10|
|    109|  1|         2020-03-03|           null|
|    109|  2|         2020-03-08|              5|
|    120|  1|         2020-03-18|           null|
+-------+---+-------------------+---------------+



In [263]:
df_output= df_td.withColumn("Status", when(df_td.time_intertweet <= "7","Active").when(df_td.time_intertweet.isNull() ,"").otherwise("In-Active"))

In [261]:
df_output.filter(df_output.Status == "Active").selectExpr('user_id').show()

+-------+
|user_id|
+-------+
|    109|
+-------+

